In [12]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

In [6]:
headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    }

url = 'https://www.zillow.com/rockville-md/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-77.36944914453125%2C%22east%22%3A-76.92450285546875%2C%22south%22%3A39.02324660629151%2C%22north%22%3A39.18469662654337%7D%2C%22usersSearchTerm%22%3A%22Rockville%2C%20MD%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A33714%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D'
req = requests.get(url, headers=headers)

b'<!DOCTYPE html><html lang="en"><head><link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Open+Sans:400,600,700&amp;display=swap"/><style type="text/css">@font-face{font-display:swap;font-family:Ivar Headline;font-style:normal;font-weight:600;src:url(//www.zillowstatic.com/static-zsg/LATEST/static-zsg/zsg/z-fonts/ivar/IvarHeadline-SemiBold-extended.woff2) format(&quot;woff2&quot;),url(//www.zillowstatic.com/static-zsg/LATEST/static-zsg/zsg/z-fonts/ivar/IvarHeadline-SemiBold-extended.woff) format(&quot;woff&quot;);unicode-range:u+0000-001f,u+0080-200f,u+2020-faff,u+fb10-ffff}@font-face{font-display:swap;font-family:Ivar Headline;font-style:normal;font-weight:600;src:url(//www.zillowstatic.com/static-zsg/LATEST/static-zsg/zsg/z-fonts/ivar/IvarHeadline-SemiBold-core.woff2) format(&quot;woff2&quot;),url(//www.zillowstatic.com/static-zsg/LATEST/static-zsg/zsg/z-fonts/ivar/IvarHeadline-SemiBold-core.woff) format(&quot;woff&quot;);unicode-range:u+0020-007f,u+201?,u+fb0?}</st

In [45]:
soup = bs(req.content)
data = soup.find('script', {'id':'__NEXT_DATA__'})
data = json.loads(data.text)
data = data['props']['pageProps']['searchPageState']['cat1']['searchResults']['listResults'] # This is one page of results

In [76]:
df = pd.json_normalize(data)
columns = ['hdpData.homeInfo.zpid','addressStreet', 'hdpData.homeInfo.unit', 'addressCity', 'addressState', 'addressZipcode','hdpData.homeInfo.homeType',  'beds', 'baths', 'area', 'pgapt',
           'hdpData.homeInfo.price', 'hdpData.homeInfo.homeStatus', 'hdpData.homeInfo.priceForHDP', 'hdpData.homeInfo.taxAssessedValue', 'zestimate', 'hdpData.homeInfo.rentZestimate',
           'hdpData.homeInfo.daysOnZillow','hdpData.homeInfo.isPreforeclosureAuction',  'brokerName', 'latLong.latitude', 'latLong.longitude',  
           'hasOpenHouse', 'openHouseStartDate', 'openHouseEndDate', 'builderName',
           'hdpData.homeInfo.listing_sub_type.is_newHome']

df = df[columns]
columns = [col.split('.')[-1] for col in columns]

df.columns = columns
df
df['StreetNumber'] = df['addressStreet'].map(lambda x: x.split(' ')[0])
df['Street'] = df['addressStreet'].map(lambda x:' '.join(x.split(' ')[1:]))

df['unit'].str.len()

0     NaN
1     NaN
2     5.0
3     NaN
4     NaN
5     NaN
6     5.0
7     7.0
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    7.0
17    NaN
18    4.0
19    NaN
20    NaN
21    NaN
22    NaN
23    7.0
24    NaN
25    7.0
26    NaN
27    NaN
28    NaN
29    NaN
30    NaN
31    NaN
32    NaN
33    7.0
34    NaN
35    NaN
36    NaN
37    NaN
38    NaN
39    NaN
40    NaN
Name: unit, dtype: float64